In [1]:
# import statements
import os, sys
# sys.path.append('../')
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numbers as np
import requests
from pathlib import Path
import json
import pickle
import folium
from branca.element import Figure
import seaborn as sns
import datetime
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# devices in FORT PORTAL
devices = ["aq_g5_36", "aq_g533", "aq_g530", "aq_g528", "aq_10", "aq_g534","aq_g527","aq_g526" ,"aq_g535"]

# query device details and data 
kabale_devices = []
for device in devices:
    temp = {}
    try:
        api_url = f'https://api.airqo.net/api/v1/devices?tenant=airqo&name={device}'
        results = requests.get(api_url, verify=False)
        device_details = results.json()['devices']

        temp['name'] = device_details[0]['name']
        temp['id'] = device_details[0]['device_number']
        temp['lat'] = device_details[0]['latitude']
        temp['long'] = device_details[0]['longitude']

        # append to overall result
        kabale_devices.append(temp)
    except Exception as ex:
        print("Devices Url returned an error: " + str(ex))
        device_details = {}
    

In [3]:
kabale_devices

[{'name': 'aq_g5_36', 'id': 1429567, 'lat': 0.6888, 'long': 30.26955},
 {'name': 'aq_g533', 'id': 1379966, 'lat': 0.64726, 'long': 30.26844},
 {'name': 'aq_g530', 'id': 1375499, 'lat': 0.6569, 'long': 30.27896},
 {'name': 'aq_g528', 'id': 1375496, 'lat': 0.65561, 'long': 30.27199},
 {'name': 'aq_10', 'id': 676000, 'lat': 0.657624, 'long': 30.26763},
 {'name': 'aq_g534', 'id': 1379967, 'lat': 0.6301, 'long': 30.2734},
 {'name': 'aq_g527', 'id': 1375495, 'lat': 0.66668, 'long': 30.28556},
 {'name': 'aq_g526', 'id': 1375494, 'lat': 0.6237, 'long': 30.25565},
 {'name': 'aq_g535', 'id': 1379969, 'lat': 0.68736, 'long': 30.24734}]

In [4]:
kabale_devices_df = pd.DataFrame(kabale_devices)
kabale_devices_df

,name,id,lat,long
0,aq_g5_36,1429567,0.688800,30.26955
1,aq_g533,1379966,0.647260,30.26844
2,aq_g530,1375499,0.656900,30.27896
3,aq_g528,1375496,0.655610,30.27199
4,aq_10,676000,0.657624,30.26763
5,aq_g534,1379967,0.630100,30.27340
6,aq_g527,1375495,0.666680,30.28556
7,aq_g526,1375494,0.623700,30.25565
8,aq_g535,1379969,0.687360,30.24734


In [5]:
# save devices

kabale_devices_df.to_csv('D:\PYTHON\WORK-2022\spatial-model/fort_portal/Fort_portal_devices.csv')

In [6]:
long =kabale_devices_df['long'].mean()
lat = kabale_devices_df['lat'].mean()

In [7]:
# draw kabale airqloud map

fig=Figure(width=600,height=700)
kabale_map=folium.Map(width=600,height=700,location=[lat,long],zoom_start=13,min_zoom=8,max_zoom=14)
fig.add_child(kabale_map)

for index, row in kabale_devices_df.iterrows():
    label = f'{row["name"]}'
    folium.Marker(location=[row['lat'], row['long']], 
                      popup=folium.Popup(html=label, show=True, sticky=True), 
                      icon=folium.Icon(color='blue'),
                      tooltip=label).add_to(kabale_map)
kabale_map

In [8]:
result=[]

for device in devices:

    api_url = f'https://api.airqo.net/api/v1/devices/events?tenant=airqo&device={device}&endTime={date.today()}'
    response = requests.get(api_url, verify=False)
    measurements = response.json()['measurements']


    for value in measurements:
        # print(value.get('pm2_5') and value.get('pm2_5').get('value"'))
        transformed_event = {
            'device_name': value.get('device'),
            'device_number': value.get('device_number'),
            'pm2_5': value.get('pm2_5') and value.get('pm2_5').get('calibratedValue'),
            'time': value.get('time')
        }
        # print(transformed_event)

        # if transformed_event['time']:
        #     transformed_event['time'] = transformed_event['time'].isoformat()

        result.append(transformed_event)



In [9]:
data = pd.DataFrame(result)
data.head()

,device_name,device_number,pm2_5,time
0,aq_g5_36,1429567,20.0791,2022-07-14T22:00:00.000Z
1,aq_g5_36,1429567,18.4292,2022-07-14T21:00:00.000Z
2,aq_g5_36,1429567,20.0652,2022-07-14T20:00:00.000Z
3,aq_g5_36,1429567,48.3041,2022-07-14T19:00:00.000Z
4,aq_g5_36,1429567,139.4756,2022-07-14T18:00:00.000Z


In [10]:
data.shape

(3438, 4)

In [11]:
# save measurements

data.to_csv('D:\PYTHON\WORK-2022\spatial-model/fort_portal/Fort_devices_measurements.csv')

In [12]:
# FORT device details and measurements

devices_df = pd.read_csv('D:\PYTHON\WORK-2022\spatial-model/fort_portal/Fort_portal_devices.csv')
measurements_df = pd.read_csv('D:\PYTHON\WORK-2022\spatial-model/fort_portal/Fort_devices_measurements.csv')

kabale_data = measurements_df

In [13]:
kabale_data.head()

,Unnamed: 0,device_name,device_number,pm2_5,time
0,0,aq_g5_36,1429567,20.0791,2022-07-14T22:00:00.000Z
1,1,aq_g5_36,1429567,18.4292,2022-07-14T21:00:00.000Z
2,2,aq_g5_36,1429567,20.0652,2022-07-14T20:00:00.000Z
3,3,aq_g5_36,1429567,48.3041,2022-07-14T19:00:00.000Z
4,4,aq_g5_36,1429567,139.4756,2022-07-14T18:00:00.000Z


In [14]:
kabale_data['latitude'] = kabale_data['device_name'].map(devices_df.set_index('name')['lat'])
kabale_data['longtitude'] = kabale_data['device_name'].map(devices_df.set_index('name')['long'])

In [15]:
kabale_data.head()

,Unnamed: 0,device_name,device_number,pm2_5,time,latitude,longtitude
0,0,aq_g5_36,1429567,20.0791,2022-07-14T22:00:00.000Z,0.6888,30.26955
1,1,aq_g5_36,1429567,18.4292,2022-07-14T21:00:00.000Z,0.6888,30.26955
2,2,aq_g5_36,1429567,20.0652,2022-07-14T20:00:00.000Z,0.6888,30.26955
3,3,aq_g5_36,1429567,48.3041,2022-07-14T19:00:00.000Z,0.6888,30.26955
4,4,aq_g5_36,1429567,139.4756,2022-07-14T18:00:00.000Z,0.6888,30.26955


In [16]:
kabale_data.drop(columns={'Unnamed: 0'}, inplace=True)

In [17]:
# save kabale final dataframe
kabale_data.to_csv('D:\PYTHON\WORK-2022\spatial-model/fort_portal/FORT_FINAL_data.csv')

In [18]:
kabale_data.shape

(3438, 6)

In [19]:
kabale_data.head()

,device_name,device_number,pm2_5,time,latitude,longtitude
0,aq_g5_36,1429567,20.0791,2022-07-14T22:00:00.000Z,0.6888,30.26955
1,aq_g5_36,1429567,18.4292,2022-07-14T21:00:00.000Z,0.6888,30.26955
2,aq_g5_36,1429567,20.0652,2022-07-14T20:00:00.000Z,0.6888,30.26955
3,aq_g5_36,1429567,48.3041,2022-07-14T19:00:00.000Z,0.6888,30.26955
4,aq_g5_36,1429567,139.4756,2022-07-14T18:00:00.000Z,0.6888,30.26955


In [20]:
min(kabale_data['time'])

'2022-06-15T00:00:00.000Z'

In [21]:
max(kabale_data['time'])

'2022-07-14T22:00:00.000Z'

In [22]:
kabale_data.groupby('device_name').count()['pm2_5']

device_name
aq_10       673
aq_g526     160
aq_g527     713
aq_g528      10
aq_g530      11
aq_g533       5
aq_g534     714
aq_g535     446
aq_g5_36    706
Name: pm2_5, dtype: int64